In [1]:
from trustworthai.utils.losses_and_metrics.dice_loss import DiceLossWithWeightedEmptySlices
from trustworthai.utils.losses_and_metrics.dice_loss_metric import DiceLossMetric, SsnDiceMeanMetricWrapper
from trustworthai.utils.losses_and_metrics.ssn_losses import SSNCombinedDiceXentLoss

In [2]:
from trustworthai.utils.augmentation.augmentation_pipelines import get_transforms, none_transform

In [3]:
from trustworthai.utils.data_preprep.dataset_pipelines import load_data

running on local machine!


In [4]:
from trustworthai.utils.fitting_and_inference.fitters.basic_lightning_fitter import StandardLitModelWrapper

In [5]:
from trustworthai.utils.fitting_and_inference.get_trainer import get_trainer

In [6]:
from trustworthai.models.core_models.Hypermapp3r import HyperMapp3r
from trustworthai.models.stochastic_wrappers.ssn.ssn import SSN

In [7]:
from torchinfo import summary
import torch

In [8]:
# get the 2d axial slice dataloaders
train_dl, val_dl, test_dl = load_data(
    dataset="chal", 
    test_proportion=0.15, 
    validation_proportion=0.15,
    seed=3407,
    empty_proportion_retained=1,#0.1,
    batch_size=12,
    dataloader2d_only=True,
    cross_validate=True,
    cv_split=5
)

In [9]:
train_dl.dataset[0][0].sum(), val_dl.dataset[0][0].sum()

(tensor(16001.1396), tensor(18703.8047))

In [10]:
len(train_dl.dataset) + len(val_dl.dataset)

3580

In [11]:
len(train_dl), len(val_dl), len(test_dl)

(224, 75, 75)

x, y = next(iter(train_dl))

In [14]:
encoder_sizes=[16,32,64,128,256]
base_model = HyperMapp3r(
    dropout_p=0.,
    encoder_sizes=encoder_sizes,
    inchannels=3,
    outchannels=encoder_sizes[1],
)

In [15]:
model_raw = SSN(
    base_model=base_model,
    rank=15,
    diagonal=False,
    epsilon=1e-5,
    intermediate_channels=base_model.output_channels,
    out_channels=2,
    dims=2
    ).cuda()

In [16]:
model_raw.mean_l

Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1), padding=same)